# Run LLM locally

In [5]:
from gpt4all import GPT4All

In [7]:
!pip install ctransformers>=0.2.24

In [8]:
from ctransformers import AutoModelForCausalLM

c:\Users\shicong.cao\AppData\Local\anaconda3\envs\gpt\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
from langchain.llms import CTransformers
llm = CTransformers(model="morgendigital/Llama-2-13b-chat-german-GGUF", model_file="Llama-2-13b-chat-german-GGUF.q5_K_M.bin", model_type="llama")

Fetching 0 files: 0it [00:00, ?it/s]
Fetching 1 files: 100%|██████████| 1/1 [00:00<?, ?it/s]


In [12]:
from langchain import PromptTemplate,  LLMChain

template = """
              Sie sind ein professioneller Assistent. Bitte antworten Sie vollständig auf Deutsch
              ```{text}```
           """

prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

In [13]:
text1 = """
Welche Anforderungen werden an Nassräume im Hinblick auf die Barrierefreiheit in einem Krankenhaus gestellt?
"""

print(llm_chain.run(text1))

 In Krankenhäusern werden Nassräume, wie Badezimmer und Duschkabinen, aufgrund der Barrierefreiheitsanforderungen entsprechend den Richtlinien geplant und ausgestattet. Diese Anforderungen richten sich nach der Verordnung über die barrierefreie Gestaltung von Gebäuden (Barrierefreiheitsverordnung - BVG).

Die wichtigsten Barrierefreiheitsanforderungen für Nassräume in Krankenhäusern sind:

1. Tauglichkeit für Menschen mit körperlichen Beeinträchtigungen oder Sehbeeinträchtigungen: Der Raum muss so gestaltet sein, dass er jeder Person mit körperlicher Beeinträchtigung oder Sehbeeinträchtigung zugänglich ist. Zum Beispiel sollten Badewannen einen niedrigen Ein- und Ausstieg bieten.
2. Anpassbare Wannenhöhe: Die Badewanne muss so geplant sein, dass sie an verschiedene Körpergrößen angepasst werden kann und Personen mit körperlicher Beeinträchtigung


In [14]:
!pip install gradio

In [15]:
import os
import io
from IPython.display import Image, display, HTML
from PIL import Image
import base64 
import requests, json

In [16]:
import gradio as gr

In [17]:
def answer(input):
    output = llm_chain.run(input)
    return output

server_port = int(os.environ.get('PORT1', 7860))
    
gr.close_all()
demo = gr.Interface(fn=answer, 
                    inputs=[gr.Textbox(label="Your Question", lines=6)],
                    outputs=[gr.Textbox(label="Result", lines=3)],
                    title="Question Answer in German",
                   )
demo.launch(share=True, server_port=server_port)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://49460920637b5b548a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# PDF retrieval

## Load and Split the document

In [42]:
model = GPT4All("C:\\Users\\shicong.cao\\Documents\\llama-2-13b-chat.ggmlv3.q3_K_S.bin")

Found model file at  C:\\Users\\shicong.cao\\Documents\\llama-2-13b-chat.ggmlv3.q3_K_S.bin


In [7]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader('Q:\daten\982-BIM_int\_MA\Cao\Langchain\DIN-13080_06-2016_ENGLISCH.pdf')

doc= loader.load()

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1500,
                chunk_overlap=100,
)

splits = text_splitter.split_documents(doc)

In [9]:
len(splits)

137

In [10]:
print(splits[25])

page_content='Protection)  \nBioStoffV  Biost offverordnung  (Ordinance on Biological Agents)  \nBMUB  Bundesministerium für Umwelt, Naturschutz, Bau und Reaktorsicherheit  (Federal \nMinistry for Environment, Nature Conservation, Building and Nuclear Safety)  \nBOS  Behörden und Organisationen mit Sicherheitsa ufgaben  (Authorities and organizations \nwith safety tasks)  \nBSG  Bundessozialgericht  (Federal Social Court)  \n  \nCT computer tomograph  \nCTG  cardiotocography  \n  \nDAB  Deutsches Arzneibuch  (German Pharmacopoeia)  \nDAV  Durchgangsarztverfahren  (Procedure involving  trauma specialists accredited by the \nemployer ʼs liability insurance associations)  \nDG Deutsche Gesellschaft  (German Society)  \nDGK  Deutsche Gesellschaft für Kardiologie  (German Society of Cardiology)  \nDGKH  Deutsche Gesellschaft für Krankenhaushygiene  (Germ an Society of Hospital Hygiene)  \nDGUV  Deutsche Gesetzliche Unfallversicherung  (German Social Accident Insurance)  \nDIVI  Deutsche Int

## Build Vectorbase

In [11]:
from langchain.embeddings import GPT4AllEmbeddings
embedding = GPT4AllEmbeddings()

Found model file at  C:\\\\Users\\\\shicong.cao\\\\.cache\\\\gpt4all\\ggml-all-MiniLM-L6-v2-f16.bin


In [12]:
from langchain.vectorstores import Chroma

In [13]:
persist_directory = 'Q:\daten\982-BIM_int\_MA\Cao\Langchain\chroma'

In [14]:
vectordb= Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [15]:
print(vectordb._collection.count())

137


## Retrieve relevant pieces from Vectorbase

In [21]:
question = "Is there requirement for wet rooms in terms of barrier free?"

In [26]:
docs = vectordb.similarity_search(question, k=5)

In [27]:
docs[].page_content

'multiresistenten -gramnegativen Stäbchen  (Hygiene measures for the \ncontrol or colonization with multiresistant gram -negative rods)  \nKRINKO, Infektionsprävention im Rahmen d er Pflege und Behandlung von \nPatienten mit übertragbaren Krankheiten, Abschnitt 3.3 : Räumliche \nUnterbringung (Infection prevention during nursing care and treatment \nof patients with transmissible diseases, Section 3.3: Spatial \naccommodation)  \n  \nPatient roo ms \nincluding sanitary room  \n   \nA General  \nAppropriate consideration of clearance . \n B Hygiene  \nAccording to the recommendations of the DGKH, it should be possible for \nabout half of the rooms to be used as isolation rooms.  \nWet room immediately accessible from the room . \n \n   B55EB1B3C7662F79D1B59483A53B9F2F82C98BEEB7939B8362DE64BB99B626987BF0460D90BD5ED1D09C61AC5C4DC88EA0069FFA18F73874A12A2EBB2ECA67CC313AF49141000AFE482C08060EBD23778CEF0775AAA4BF94E3726FD05D94523CEC09BC80B38ED0B3A35CA131A397AD3F87\nNormen-Download-Beuth-Heinl

In [28]:
docs[1].page_content

'therapy pool,  \ncold therapy,  \ncold chambers,  \nrest room  \nTreatment room   \nA General  \nRooms for external service providers are to be marked , if applicable.  \nAppropriate areas for patient changing rooms and waiting areas . \nFrom the organizational point of view, such supportive treatments are \nperformed on the ward.  \n \nC Occupational health and safety  \nConsider areas for staff (break r oom, lockers for valuables , wet rooms) . \n \nE Bibliography  \nDIN 19643 (all parts), Treatment of water of swimming pools and baths  \nKRINKO, Anforderungen der Hygiene an die funktionelle und bauliche \nGestaltung von Einrichtungen in der Physiotherapie (Physikalisc he \nTherapie) (Hygiene requirements for the functional and architectural \ndesign  of physiotherapy installations (physical therapy))  \nVDI 2089 Blatt  1, RTL -Anlagen für Schwimmhallen und Bäder ( Ventilation \nand air conditioning installations for swimming baths a nd pools ) \n   B55EB1B3C7662F79D1B59483A53B9F2F8

In [29]:
docs[].page_content

'systems    \nA General  \nThe location of the technical area s shall be allocated to the main users \nand consumers, e.g. air conditioning to 1.09, in order to avoid long installation distances.  \n B Hygiene  \nWhen planning ventilation and air conditioning systems for medically \nused rooms/f acilities, the hospital hygienist defines the room classes and \nventilation/air conditioning requirements for hygiene (e.g. positive/negative pressure).  \nRequirements according to the relevant legislation, guidelines and body of rules . \nHeat exchangers are to  be planned in an air -cooled or adiabatic (dry) \nmanner. Adequate outdoor positions are to be planned for placement \n(noise). If wet heat exchangers are used, special consideration is to be \ngiven to the site selection (indoor/outdoor) and to the cooling water quality (microbiological quality of circulation cooling water).  \n E Bibliography  \nDIN  1946 -4, Ventilation and air conditioning  \n\uf8e7 Ventilation in buildings \nand

In [30]:
docs[3].page_content

'DIN  13080:2016- 06 \n34 FUNCTIONAL AREA  \nFunctional section  \nSubsection  \n- Subsubsection  Examples of rooms  \nand areas, \nexplanations of \nallocation to (  ) Planning guidance notes  \n2.06 Isolation care  \n \n \n \n \n \n   Patient rooms with \nsanitary room and \nairlock , \npatient  airlock , \nstaff airlock , \nvisitor airlock , \nsupply airlock , \nwaste disposal airlock , \n \nFurther rooms in \nanalogy to 2.00  A Gene ral \nA separate subsection is required  for patients with highly infectious \ndiseases.  \n \nB Hygiene  \nWards for patients with infections shall be arranged and designed in a \nway that prevents the further spread of diseases from one patient room to another and from  the rooms to other areas within the hospital.  \nAll access areas are to be designed with airlock s. \nSanitary room with wash basin, shower, toilet with WC with flusher - \ndisinfector, touch -free taps . \n \n2.06.01 Infectious disease  \ncare  \n \n2.06.02 Reverse isolation  \n(KMT)

In [31]:
docs[4].page_content

'DIN  13080:2016- 06 \n33 FUNCTIONAL AREA  \nFunctional section  \nSubsection  \n- Subsubsection  Examples of rooms  \nand areas, \nexplanations of \nallocation to (  ) Planning guidance notes  \n2.05.02 Neonatal care   \n \n \n \n \n \n Patient room − child \nwith parents,  \ninfant room with \nairlock , \nbreast -feeding room,  \nfood preparation,  \ncommon room  − \nparents,  \novernight stay − parents  \n \nFurther rooms in \nanalogy to 2.00  \n A General  \nArrange for access control and take precautions against baby -snatching . \n B Hygiene  \nSufficient clearance at four sides around the bed . \nAccording to the recommendations of the DGKH, it should be possible for about half of the rooms to be used as iso lation rooms.  \nChanging rooms for staff and visitors.  \n \nD Technique  \nAir conditioning system that supports hygiene requirements  \nand prevents infection risk . \n \n2.05.03 Paediatric intensive \ncare  \n \n \n \n Patient rooms,  \ncommon rooms − \nparents,  \novern

In [32]:
for doc in docs:
    print(doc.metadata)

{'page': 25, 'source': 'Q:\\daten\\982-BIM_int\\_MA\\Cao\\Langchain\\DIN-13080_06-2016_ENGLISCH.pdf'}
{'page': 23, 'source': 'Q:\\daten\\982-BIM_int\\_MA\\Cao\\Langchain\\DIN-13080_06-2016_ENGLISCH.pdf'}
{'page': 51, 'source': 'Q:\\daten\\982-BIM_int\\_MA\\Cao\\Langchain\\DIN-13080_06-2016_ENGLISCH.pdf'}
{'page': 33, 'source': 'Q:\\daten\\982-BIM_int\\_MA\\Cao\\Langchain\\DIN-13080_06-2016_ENGLISCH.pdf'}
{'page': 32, 'source': 'Q:\\daten\\982-BIM_int\\_MA\\Cao\\Langchain\\DIN-13080_06-2016_ENGLISCH.pdf'}
